In [ ]:
# What features of used cars increase the prices of the cars?
# The assumption is that the sale of a higher priced car nets more money for a used car dealer. There is the risk that this may not be true.
# Based on a Kaggle dataset of used cars, determine which features are most correlated with car prices. www.kaggle.com

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
from scipy.linalg import svd
from scipy.stats import zscore
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
cars = pd.read_csv('data/vehicles.csv')

In [ ]:
cars.info()
print(cars.head())
print(cars.tail())
cars.describe()

In [ ]:
cars.isnull().sum()

In [ ]:
# The VIN contains a lot of information about a car not included in the dataset.
# The VIN numbers varied depending in the manufacturer before 1981.
# Dropping rows before 1981 to allow use of VIN and
# because used car dealers are not likely to see many cars that old.
# Also assuming there is not a big market for antique cars although they can bring very high prices.
rows_to_drop = cars[cars['year'] < 1981].index
cars_cleaned = cars.drop(rows_to_drop)

# split VIN into it's components and keep columns not already included in data
cars_cleaned['assembly_country'] = cars_cleaned['VIN'].str[:1] # final assembly country
cars_cleaned['type_or_division'] = cars_cleaned['VIN'].str[2:3] # type or manufacturing division
cars_cleaned['attributes'] = cars_cleaned['VIN'].str[3:7] # attributes, such as model, body style, engine type, transmission type etc.
cars_cleaned['assembly_plant'] = cars_cleaned['VIN'].str[10:11] # assembly plant
vehicles = cars_cleaned

In [ ]:
# Convert text to numbers to allow correlation calculations

# not working with these datatypes in the interest of time.
# dropping these because one-hot encoding will increase features a lot:
# region, model, state. I do think these are relevant features.

# Ignore Future Warnings
warnings.filterwarnings("ignore", category=FutureWarning)
vehicles.drop(columns=(['region', 'model', 'state']), axis=1, inplace=True)

# Assigning numeric values to allow correlation. This may introduce some bias
# because it requires making assumptions about what values are better than others.
# However, it prevents adding many features if one-hot encoding were used.
vehicles['condition#'] = vehicles['condition'].replace({'new':5, 'excellent':4, 'like new':3, 'good':2, 'fair':1, 'salvage':0})
vehicles['cylinders#'] = vehicles['cylinders'].replace({'other':0, '3 cylinders':3, '4 cylinders':4, '5 cylinders':5, '6 cylinders':6, '8 cylinders':8, '10 cylinders':10, '12 cylinders':12})
vehicles['title_status#'] = vehicles['title_status'].replace({'clean':5, 'lien':4, 'rebuilt':3, 'missing':2, 'salvage':0, 'parts only':1})
vehicles['size#'] = vehicles['size'].replace({'full-size':3, 'mid-size':2, 'compact':1})

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`.

In [ ]:
# drop duplicates
vehicles.drop_duplicates(inplace=True)
# drop rows with NAN
vehicles.dropna(inplace=True)
# making assumption that all cars have some value even as scrap or parts
rows_to_drop = vehicles[vehicles['price'] == 0].index
vehicles = vehicles.drop(rows_to_drop)
vehicles['year'] = vehicles['year'].astype(int) # year is an int not a float
# drop erroneous price row
rows_to_drop = vehicles[vehicles['price'] == 12345678].index
vehicles.drop(rows_to_drop, inplace=True)
vehicles.info()

In [ ]:
# One-hot encoding
# One-hot encode fuel because unsure what the order is and only adds 3 features
cars_cleaned2 = pd.get_dummies(vehicles, columns=['fuel'])
# One-hot encode transmission because unsure what order is and only adds 2 features
cars_cleaned3 = pd.get_dummies(cars_cleaned2, columns=['transmission'])
# One-hot encode drive because unsure what order is and only adds 2 features
cars_cleaned4 = pd.get_dummies(cars_cleaned3, columns=['drive'])
# One-hot encode cylinders because 0 cylinders do not plot well with other cylinder values
cars_cleaned5 = pd.get_dummies(cars_cleaned4, columns=['cylinders#'])

# drop the id feature assuming it has no correlation with price
cars_cleaned5.drop('id', axis=1, inplace=True)
vehicles = cars_cleaned5

# Set one-hot endoding from True/False to 1/0
vehicles.replace(({True:1, False:0}), inplace=True)
vehicles

In [ ]:
vehicles.info()

In [ ]:
# remove outliers with Z-score >= 3
vehicles_numeric = vehicles.select_dtypes(include=[np.number])
for i in vehicles_numeric.columns:
  vehicles_numeric['Z_score'] = zscore(vehicles_numeric[i])
  vehicles_cleaned = vehicles_numeric[np.abs(vehicles_numeric['Z_score']) <= 3].copy()
vehicles_cleaned.drop('Z_score', axis=1, inplace=True)
vehicles_cleaned

In [ ]:
vehicles_cleaned.corr()
# all columns have some correlation with price.
vehicles_cleaned.corr()

In [ ]:
price_corr = (vehicles_cleaned.corr()['price']).abs()
price_corr.sort_values(ascending=False)
# Note that using the number of cylinders as the value correlates better
# with price than if numbers assigned based on guess at most popular
# number of cylinders (e.g. 6 cylinders preferred over 4).
# It is not surprising that year and odometer have a correlation of about 0.5.
# If the correlation were higher, I might eliminate the odometer as a feature.

In [ ]:
# feature selection
def svd_norm(X): #normalizes data & runs PCA
  x_norm =( X - X.mean())/X.std()
  U, sigma, VT = svd(x_norm, full_matrices=False)
  Sigma = np.diag(sigma)
  return U, Sigma, VT

U, Sigma, VT = svd_norm(vehicles_numeric)
sig = np.diag(Sigma)
sigsum = sig.cumsum()/sig.sum()
plt.plot(sigsum, 'go-')
plt.grid(True)
plt.show()
plt.plot(sig, 'ro-')
# r = 6 features with excellent correlation to price. Could add 3 more features.
r = 6
# Can run with the most important features or with all the features
#car_short = vehicles_numeric[['price', 'year', 'fuel_diesel', 'fuel_gas', 'drive_fwd', 'cylinders#_8.0', 'drive_4wd']]
car_short = vehicles_numeric

In [ ]:
def pca(X, r): #projects data
    x_norm =( X - X.mean())/X.std()
    U, sigma, VT = svd(x_norm, full_matrices=False)
    Sigma = np.diag(sigma)
    Ur = U[:, :r]
    Sigma_r = Sigma[:r, :r]
    return pd.DataFrame(Ur @ Sigma_r, columns=[f'pca_{i}' for i in range(1, r + 1)])
pca(vehicles_numeric, r)

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
car_short

In [ ]:
# Split the target from the features
X = car_short.drop('price', axis=1)
y = car_short['price']
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# fit linear
X_train_feat = pd.DataFrame(X_train['year'])
X_test_feat = pd.DataFrame(X_test['year'])
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train_feat, y_train)

# year appears curved relative to price. Degree 3 gives lower MSE than degree 2.
# However, if fit_intercept=True, linear produces the lowest MSE.
# Making the assumption there are few buyers for classic cars although the prices are higher.
# Comfortable underestimating the prices of older cars, including negative values, in giving advice
# to used car dealers because they will make fewer sales and therefore less money on classic cars.
# Classic cars are a different market.
# Proceeding with the linear model and fit_intercept=True.
poly_transform = PolynomialFeatures(degree=3, include_bias=False)
X_poly_year = poly_transform.fit_transform(X_train_feat)
lr_3 = LinearRegression(fit_intercept=True)
lr_3.fit(X_poly_year, y_train)
y_pred_poly = lr_3.predict(X_poly_year)

# plot predictions over real data
X_grid = np.arange(min(X_train_feat.values), max(X_train_feat.values), 1)
X_grid = X_grid.reshape((len(X_grid), 1))
plt.scatter(X_train_feat, y_train)
plt.scatter(X_train_feat, y_pred_poly, color='red')
plt.plot(X_train_feat, lr.predict(X_train_feat), color='green')
plt.title('Linear and Polynomial (degree=3)')
plt.xlabel('Manufacturing Year')
plt.ylabel('Price ($)')
plt.legend(['Real Data', 'Polynomial (3)', 'Linear Prediction'])

mse_lin = mean_squared_error(X_test['year'], lr.predict(X_test_feat))
mae_lin = mean_absolute_error(X_test['year'], lr.predict(X_test_feat))
rmse_lin = np.sqrt(mse_lin)
r2_lin = r2_score(X_test['year'], lr.predict(X_test_feat))
print('Linear statistics')
print(f'RMSE: {rmse_lin}')
print(f'r2: {r2_lin}')
print(f'MSE: {mse_lin}')
print(f'MAE: {mae_lin}\n')

mse_poly = mean_squared_error(X_test['year'], lr_3.predict(poly_transform.fit_transform(X_test_feat)))
mae_poly = mean_absolute_error(X_test['year'], lr_3.predict(poly_transform.fit_transform(X_test_feat)))
rmse_poly = np.sqrt(mse_poly)
r2_poly = r2_score(X_test['year'], lr_3.predict(poly_transform.fit_transform(X_test_feat)))
print('Polynomial 3 statistics')
print(f'RMSE: {rmse_poly}')
print(f'r2: {r2_poly}')
print(f'MSE: {mse_poly}')
print(f'MAE: {mae_poly}')

In [ ]:
X_train

In [ ]:
# fit linear
X_train_feat = pd.DataFrame(X_train['fuel_diesel'])
X_test_feat = pd.DataFrame(X_test['fuel_diesel'])
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train_feat, y_train)

# plot prediction over real data
plt.scatter(X_train_feat, y_train)
plt.plot(X_train_feat, lr.predict(X_train_feat), color='green')
plt.title('Linear Regression')
plt.xlabel('Diesel Fuel')
plt.ylabel('Price')
plt.legend(['Real Data', 'Linear Prediction'])

# Calculate statistics
mse = mean_squared_error(X_test_feat, lr.predict(X_test_feat))
mae = mean_absolute_error(X_test_feat, lr.predict(X_test_feat))
rmse = np.sqrt(mse_lin)
r2 = r2_score(X_test_feat, lr.predict(X_test_feat))

# Print statistics
print(f'RMSE: {mse}')
print(f'r2: {r2}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')

In [ ]:
# fit linear
X_train_feat = pd.DataFrame(X_train['fuel_gas'])
X_test_feat = pd.DataFrame(X_test['fuel_gas'])
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train_feat, y_train)

# plot prediction over real data
#plt.scatter(X_train_feat, y_train)
#plt.plot(X_train_feat, lr.predict(X_train_feat), color='green')
#plt.title('Linear Regression')
#plt.xlabel('Gas Fuel')
#plt.ylabel('Price')
#plt.legend(['Real Data', 'Linear Prediction'])

# Calculate statistics
mse = mean_squared_error(X_test_feat, lr.predict(X_test_feat))
mae = mean_absolute_error(X_test_feat, lr.predict(X_test_feat))
rmse = np.sqrt(mse_lin)
r2 = r2_score(X_test_feat, lr.predict(X_test_feat))

# Print statistics
print(f'RMSE: {mse}')
print(f'r2: {r2}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')

In [ ]:
# fit linear
X_train_feat = pd.DataFrame(X_train['drive_fwd'])
X_test_feat = pd.DataFrame(X_test['drive_fwd'])
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train_feat, y_train)

# plot prediction over real data
#plt.scatter(X_train_feat, y_train)
#plt.plot(X_train_feat, lr.predict(X_train_feat), color='green')
#plt.title('Linear Regression')
#plt.xlabel('Front Wheel Drive')
#plt.ylabel('Price')
#plt.legend(['Real Data', 'Linear Prediction'])

# Calculate statistics
mse = mean_squared_error(X_test_feat, lr.predict(X_test_feat))
mae = mean_absolute_error(X_test_feat, lr.predict(X_test_feat))
rmse = np.sqrt(mse_lin)
r2 = r2_score(X_test_feat, lr.predict(X_test_feat))

# Print statistics
print(f'RMSE: {mse}')
print(f'r2: {r2}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')

In [ ]:
# fit linear
X_train_feat = pd.DataFrame(X_train['cylinders#_8.0'])
X_test_feat = pd.DataFrame(X_test['cylinders#_8.0'])
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train_feat, y_train)

# plot prediction over real data
#plt.scatter(X_train_feat, y_train)
#plt.plot(X_train_feat, lr.predict(X_train_feat), color='green')
#plt.title('Linear Regression')
#plt.xlabel('8 Cylinders')
#plt.ylabel('Price')
#plt.legend(['Real Data', 'Linear Prediction'])

# Calculate statistics
mse = mean_squared_error(X_test_feat, lr.predict(X_test_feat))
mae = mean_absolute_error(X_test_feat, lr.predict(X_test_feat))
rmse = np.sqrt(mse_lin)
r2 = r2_score(X_test_feat, lr.predict(X_test_feat))

# Print statistics
print(f'RMSE: {mse}')
print(f'r2: {r2}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')

In [ ]:
# fit linear
X_train_feat = pd.DataFrame(X_train['drive_4wd'])
X_test_feat = pd.DataFrame(X_test['drive_4wd'])
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train_feat, y_train)

# plot prediction over real data
#plt.scatter(X_train_feat, y_train)
#plt.plot(X_train_feat, lr.predict(X_train_feat), color='green')
#plt.title('Linear Regression')
#plt.xlabel('4-Wheel Drive')
#plt.ylabel('Price')
#plt.legend(['Real Data', 'Linear Prediction'])

# Calculate statistics
mse = mean_squared_error(X_test_feat, lr.predict(X_test_feat))
mae = mean_absolute_error(X_test_feat, lr.predict(X_test_feat))
rmse = np.sqrt(mse_lin)
r2 = r2_score(X_test_feat, lr.predict(X_test_feat))

# Plot statistics
print(f'RMSE: {mse}')
print(f'r2: {r2}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')

In [ ]:
# Split dataset into train, dev, and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=0.1, random_state=42)

# Scale and fit the data
pipe = Pipeline([
  ('scaler', StandardScaler()),
  ('ridge', Ridge())
])
scaled_pipe = pipe.fit(X_train, y_train)
train_preds = scaled_pipe.predict(X_train)
test_preds = scaled_pipe.predict(X_test)

train_mse = mean_squared_error(y_train, train_preds)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, train_preds)
train_r2 = r2_score(y_train, train_preds)

test_mse = mean_squared_error(y_test, test_preds)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_preds)
test_r2 = r2_score(y_test, test_preds)

print('Training statistics')
print(f'RMSE: {train_rmse}')
print(f'r2: {train_r2}')
print(f'MSE: {train_mse}')
print(f'MAE: {train_mae}\n')

print('Test statistics')
print(f'RMSE: {test_rmse}')
print(f'r2: {test_r2}')
print(f'MSE: {test_mse}')
print(f'MAE: {test_mae}')
# The RMSE is over $9000 with just the most relevant featues. This is a large error.
# The error is reduced to over $8000 if all the data are used.
# Running data by the state and region should help
# Using the models of the cars should help also. This feature may need further cleaning.
# One-hot encoding car models increases the features too much.

In [ ]:
# Adjust the weights for each feature
coef_list = []
param_grid = {'alpha': [0.001, 1.0, 10.0, 100.0]}
ridge = Ridge()
ridge.fit(X_train, y_train)
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)
print("Best alpha found:", grid_search.best_params_['alpha'])
print("Best negative root mean squared error:", grid_search.best_score_)
grid_search_results = pd.DataFrame(grid_search.cv_results_)
print("Cross-validation scores:\n", grid_search_results[['param_alpha', 'split0_test_score', 'rank_test_score']])
# The weights of the selected features are about equal.
# This means the selected features are equally important to determining the price of cars.